## Part 1: manipulating YAML files with OmegaConf

See the full documentation here: [https://omegaconf.readthedocs.io/en/latest/usage.html#access-and-manipulation](https://omegaconf.readthedocs.io/en/latest/usage.html#access-and-manipulation)

In [ ]:
from omegaconf import OmegaConf

In [ ]:
conf = OmegaConf.load('configs/config.yaml')

In [ ]:
print(type(conf))

In [ ]:
# Accessing values
print(conf.dataset.image.channels)
print(conf['dataset']['classes'])

In [ ]:
# Modifying a value
conf.dataset.classes = 15

## Part 2: Using Hydra

In [ ]:
%run -i main.py

In [ ]:
# Modify a flag
%run -i main.py dataset.classes=15

In [ ]:
# Add a flag
%run -i main.py +custom_param=True

In [ ]:
# Change the working directory
%run -i main.py hydra.run.dir='outputs/custom_folder'

## Part 3: Instantiating a configuration file manually

In [ ]:
from hydra import initialize, compose

In [ ]:
# Initialize Hydra
initialize('configs')

In [ ]:
cfg = compose(config_name='config')

## Part 4: Instantiating an object

In [ ]:
import tensorflow as tf
from hydra.utils import instantiate

In [ ]:
model = tf.keras.Sequential()
model.add(
    instantiate(cfg.feature_extractor)
)    

In [ ]:
# We create the other layers one by one
for l in cfg.classifier:
    model.add(
        instantiate(cfg.classifier[l])
    )

In [ ]:
model.summary()

In [ ]:
# Instantiate the large model
cfg = compose(config_name='config', overrides=['classifier=large'])

## Part 5: An example of multiple run

In [ ]:
# We need to reset HydraGlobal because we called initialize before
hydra.core.global_hydra.GlobalHydra.instance().clear()

In [ ]:
%run -i main.py -m classifier=small,large

## Part 6: Validating the configuration

In [ ]:
# Try overriding a parameter with an invalid type
%run -i main.py dataset.classes=0.5